In [6]:

def rename_documents(dataPath):

    """ dataPath : "20_newsgroups/" """
    new_path = os.getcwd()+'/'+dataPath
    every_path = list(os.walk(new_path))
    
    total = 1
    for i in range(0,len(every_path)):
        dirPath, dirName, fileNames = every_path[i]
        if(len(dirName) == 0):
            for j in fileNames:
                single_doc_loc = dirPath + '/' + j
                os.rename(single_doc_loc, dirPath + '/' + str(total))
                total+=1
    
    return     

In [ ]:
def testDataTokens(test_doc):
    test_docs = []
    
    for doc in test_doc:
        test_docs.append(tokenizeDocument(doc))
        
    return test_docs

In [7]:
def collect_documents(dataPath):

    docs = {}
    labels = []
    
    new_path = os.getcwd()+'/'+dataPath
    every_path = list(os.walk(new_path))
    
    f = 0
    for i in range(0,len(every_path)):
        dirPath, dirName, fileNames = every_path[i]
        if(len(dirName) == 0):
            for j in fileNames:
                single_doc = []
                single_doc_loc = dirPath + '/' + j
                with open(single_doc_loc, 'rb') as f:
                    single_doc.append(str(f.read()))
                docs[j] = single_doc
                labels.append(dirPath.split('/').pop())
                
    unq_labels = set(labels)
    i = 0
    mp = {}
    for x in unq_labels:
        mp[x] = i
        i+=1
    
    print(mp)
    new_labels = [mp[x] for x in labels]
    
    return docs, new_labels, mp

In [ ]:
def tokenizeDocument(docs, qt = False):
    """
        Args:
            docs (dict) : all documents
        return:
            final tokens (list) : terms for index

        0. Convert to lowercase
        1. Stop words removed
        2. Tokenize 
        3. Stemming
        4. Lemmatization
        5. Only words that starts with alphabet or digit. Front 0's removed.
    """

    stop_ws = set(stopwords.words('english'))

    ts = docs.split('\\n')
    docs = ' '.join(ts)
    ts = docs.split('\t')

    docs = ' '.join(ts)
  
    # Tokenization
    tokens = WordPunctTokenizer().tokenize(docs)
    
    ntokens = []
    tokenizer = RegexpTokenizer(r'[^_]')
    for t in tokens:
        ntokens.append(''.join(tokenizer.tokenize(t)))

    # lowercase
    tokens_lowercase = [ w.lower() for w in ntokens]
    
    #Remove Stop words
    tokens_stop  = [ w for w in tokens_lowercase if(w not in stop_ws)] 
    
#     tokens_stem = [ PorterStemmer().stem(w) for w in tokens_stop]
    
    # Lemmatization
    updated_tokens = [ WordNetLemmatizer().lemmatize(w) for w in tokens_stop]
     
    final_tokens = []
    updated_tokens = list(filter(None, updated_tokens))
    
    for updated_token in updated_tokens:
        if(updated_token.isalpha() and len(updated_token) > 1 and  d.check(updated_token)):
            final_tokens.append(updated_token)
#         else:
#             if(updated_token.isnumeric()):
#                 final_tokens.append(str(int(updated_token)))
                
    if(not qt):
        final_tokens = final_tokens[1:]  # remove b
    else:
        return final_tokens  

    return final_tokens
      


In [2]:
def calculateMean(data_points, cluster, i):
    """
        Input : cluster docs id for single cluster
    """
    vec  = 0
        
    for doc in cluster:
        vec+= data_points[doc, :]
        
    return vec/len(cluster)

In [5]:
# Objective function
def SSE(data_points, assignment, initial_cent):
    
    error = 0
    
    for c, assigned_data_pts in assignment.items():
        if(assigned_data_pts != list()):
            for data_pt in assigned_data_pts:
                error+= np.linalg.norm(data_points[data_pt, :] - initial_cent[c,:])
            
    return error

In [3]:
def clusterAssignment(data_points, k, initial_cent):

    assign = {}
    
    for i in range(0,k):
        assign[i] = []
    
    for i in range(0, data_points.shape[0]):
        best_cluster = 0
        
        dist = None
        
        for c in range(0, initial_cent.shape[0]):
            if(dist != None):
                new_dist = np.linalg.norm(data_points[i, :] - initial_cent[c, :])
                if(new_dist <= dist):
                    dist = new_dist
                    best_cluster = c
            else:
                dist = np.linalg.norm(data_points[i, :] - initial_cent[c, :])
                best_cluster = c
        
        assign[best_cluster].append(i)
                    
    return assign
   

In [10]:
def generateCentroids(data_points, k):
    
    # Randomly select 5 data points
    permute = np.random.RandomState(seed = 1).permutation(data_points)
    return permute[:5, :]

In [4]:
def k_means(data_points, k, threshold):
    
    initial_cent = generateCentroids(data_points, k)
    
    print('Intitial Centroids: ', initial_cent)
    
    t = 0
    SSE_l = []
    
    final_assignment = None
    
    while True:
        print('Iteration number: ',t+1)
        t = t+1
    
        assignment = clusterAssignment(data_points, k, initial_cent)
       
        for c, v in assignment.items():
            print('cluster no., ',c, ' # points ', len(v))
            
        sse = SSE(data_points, assignment, initial_cent)
        
        SSE_l.append(sse)
        print("SSE-", sse)
        
        new_seeds = np.zeros((k, data_points.shape[1]))
        
        for i in range(0,k):
            new_seeds[i,:] = calculateMean(data_points, assignment[i], i)
        
#         print('Updated Centroids: ', new_seeds.shape)
        
        initial_cent = new_seeds

        if(len(SSE_l) >= 2) and (abs((SSE_l[-1] - SSE_l[-2])) <= threshold):
            final_assignment = assignment
            break
        
        if(math.isnan(sse)):
            print('No')
            break
        
    print('SSE list:', SSE_l)   
    
    return final_assignment

In [11]:
def sentenceVector(w2v_model, text):
    
    sentence_vec = np.zeros((300,))
    count = 0
    text = text.split()
    
    try:
        for word in text:
            sentence_vec+= w2v_model.get_vector(word)
            count+=1
    except KeyError:
        pass
        
    return sentence_vec/count
        

In [ ]:
def confusionMatrix(z, y):
    """
        z:  predicted labels
        y : true labels
    """
    cm = np.zeros((C, C))
    
    for i in range(0, y.shape[0]):
        cm[z[i], y[i]]+=1
            
    return cm

In [ ]:
def ROC(data, proba, title = False):
    """
        Args:
            - data : test set with last column containing labels
            - proba : np array of test set likelihoods w.r.t every class 0 to c-1, OR 1 to c, where c = total classes
        Task:
            - Plot the ROC Curve for Multi class
            - No. of thresholds = 200
    """
    prb = pd.DataFrame(proba)
    org_labels = pd.DataFrame(data, columns=['class'])
    dt = pd.concat([prb, org_labels], axis = 1)
#  
    total_cls = dt['class'].unique().shape[0]
    thresholds = np.linspace(0, 1, num = 100)
    
    roc_points = pd.DataFrame(columns = ['class', 'FPR', 'TPR']) # class, fpr, tpr
    
    for cls in range(0, total_cls):
        for thres in thresholds:
            tp = 0
            fp = 0
            fn = 0
            tn = 0
            
            tp = dt.loc[(dt[cls] > thres) & (dt['class'] == cls)].shape[0]
            fp = dt.loc[dt[cls] > thres].shape[0] - tp
            
            tn = dt.loc[(dt[cls] < thres) & (dt['class'] != cls)].shape[0]
            fn = dt.loc[dt[cls] < thres].shape[0] - tn
            
            if (fp == 0):
                if(tp == 0):
                    pts = np.array([cls, 0, 0])
                else:
                    pts = np.array([cls, 0, tp/(tp+fn)])
            elif (tp == 0):
                if(fp == 0):
                    pts = np.array([cls, 0, 0])
                else:
                    pts = np.array([cls, fp/(fp+tn), 0])
            else:
                pts = np.array([cls,fp/(fp+tn), tp/(tp+fn)])
            
    
            pts = np.array([cls,fp/(fp+tn), tp/(tp+fn)])
            roc_points = roc_points.append(pd.Series(pts, index = ['class', 'FPR', 'TPR']), ignore_index = True)
    print(roc_points)
    ax = sns.lineplot(x = 'FPR', y = 'TPR', data = roc_points, hue = 'class', legend = 'full', palette = 'tab10')
    ax.set_title('ROC Curve for '+title)
    plt.plot([0,1],[0,1],'--c')
    plt.show()
    return
        
    

In [ ]:
def drawConfusionMatrix(cm, label_mapped, title = False):
    
    predict_ll = list(label_mapped.keys())
    
    hm = pd.DataFrame(cm, index = predict_ll, columns = predict_ll)
    sns.heatmap(hm, annot = True, fmt='g', cmap = "YlGnBu")
    
    plt.title('Confusion Matrix for ' + title)
    plt.xlabel('Actual')
    plt.ylabel('Predicted')
    plt.show()
    return